In [2]:
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from tensorflow.examples.tutorials.mnist import input_data
from tqdm import tqdm

In [3]:
# system parameters
n_epochs = 10000 
batch_size = 128 
lr = 0.001 # adam:learning rate
b1 = 0.9 # adam: decay of first order momentum of gradient
b2 = 0.999 # adam: decay of first order momentum of gradient
img_size = 28 # size of each image dimension
channels = 1 # number of image channels
sample_interval = 400 #interval between image samples
alpha= 10
Dim = 28*28
p_miss = 0.5
p_hint = 0.9
img_shape = (channels, img_size, img_size)
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [5]:
# Configure data loade
mnist = input_data.read_data_sets('../../MNIST_data', one_hot = True)


Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

In [7]:
""" ==================== DISCRIMINATOR ======================== """
D_W1 = xavier_init(size = [Dim*2, 256])     # Data + Hint as inputs
D_b1 = Variable(torch.zeros(256),requires_grad=True)

D_W2 = xavier_init(size = [256, 128])
D_b2 = Variable(torch.zeros(128),requires_grad=True)

D_W3 = xavier_init(size = [128, Dim])
D_b3 = Variable(torch.zeros(Dim),requires_grad=True)  

def discriminator(x, m, g, h):
    inp = m * x + (1-m) * g  # Replace missing values to the imputed values
    inputs = torch.cat([inp,h],1)  # Hint + Data Concatenate
    D_h1 = nn.relu(torch.matmul(inputs, D_W1) + D_b1)
    D_h2 = nn.relu(torch.matmul(D_h1, D_W2) + D_b2)
    D_logit = torch.matmul(D_h2, D_W3) + D_b3
    D_prob = nn.sigmoid(D_logit)  # [0,1] Probability Output
    
    return D_prob

In [8]:
""" ==================== GENERATOR ======================== """
G_W1 = xavier_init(size = [Dim*2, 256])    # Data + Mask as inputs (Random Noises are in Missing Components)
G_b1 = Variable(torch.zeros(256),requires_grad=True)

G_W2 = xavier_init(size = [256, 128])
G_b2 = Variable(torch.zeros(128),requires_grad=True)

G_W3 = xavier_init(size = [128, Dim])
G_b3 = Variable(torch.zeros(Dim),requires_grad=True)

def generator(x,z,m):
    inp = m * x + (1-m) * z  # Fill in random noise on the missing values
    inputs = torch.cat([inp,m],1)  # Mask + Data Concatenate
    G_h1 = nn.relu(torch.matmul(inputs, G_W1) + G_b1)
    G_h2 = nn.relu(torch.matmul(G_h1, G_W2) + G_b2)
    G_prob = nn.sigmoid(torch.matmul(G_h2, G_W3) + G_b3) # [0,1] normalized Output
    
    return G_prob

In [9]:
G_params = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]
D_params = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]
params = G_params + D_params

In [10]:
""" ===================== TRAINING ======================== """
def plot(samples):
    fig = plt.figure(figsize = (5,5))
    gs = gridspec.GridSpec(5,5)
    gs.update(wspace=0.05, hspace=0.05)
    
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28,28), cmap='Greys_r')
        
    return fig
def reset_grad():
    for p in params:
        if p.grad is not None:
            data = p.grad.data
            p.grad = Variable(data.new().resize_as_(data).zero_())
            
G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

def sample_Z(m, n):
    return np.random.uniform(0., 1., size = [m, n])        

# Mask Vector and Hint Vector Generation
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C
# make output file
if not os.path.exists('MNIST_Impuation_output/'):
    os.makedirs('MNIST_Impuation_output/')
i = 1
for it in tqdm(range(n_epochs)):
    
    X_mb, _ = mnist.train.next_batch(batch_size) 
    X_mb = Variable(torch.from_numpy(X_mb))
    Z_mb = sample_Z(batch_size, Dim) 
    Z_mb = Variable(torch.from_numpy(Z_mb.astype('float32')))
    M_mb = sample_M(batch_size, Dim, p_miss)
    M_mb = Variable(torch.from_numpy(M_mb.astype('float32')))
    H_mb1 = sample_M(batch_size, Dim, 1-p_hint)
    H_mb1 = Variable(torch.from_numpy(H_mb1.astype('float32')))
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    # Dicriminator forward-loss-backward-update
    G_sample = generator(X_mb,Z_mb,M_mb)
    D_sample = discriminator(X_mb, M_mb, G_sample, H_mb)
    D_loss = -torch.mean(M_mb * torch.log(D_sample + 1e-8) + (1-M_mb) * torch.log(1. - D_sample + 1e-8)) * 2
    D_loss.backward()
    D_solver.step()
    reset_grad()
    
    # Generator forward-loss-backward-update
    G_sample = generator(X_mb,Z_mb,M_mb)
    D_sample = discriminator(X_mb, M_mb, G_sample, H_mb)
    G_loss1 = -torch.mean((1-M_mb) * torch.log(D_sample + 1e-8)) / torch.mean(1-M_mb)
    MSE_train_loss = torch.mean((M_mb * X_mb - M_mb * G_sample)**2) / torch.mean(M_mb)
    MSE_test_loss = torch.mean(((1-M_mb) * X_mb - (1-M_mb)*G_sample)**2) / torch.mean(1-M_mb)
    G_loss = G_loss1  + alpha * MSE_train_loss 
    G_loss.backward()
    G_solver.step()
    reset_grad()
    
    # Print and Plot
    
 
    
        
    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('Train_loss: {:.4}'.format(MSE_train_loss))
        print('Test_loss: {:.4}'.format(MSE_test_loss))
        print()
        
        X_mb, _ = mnist.train.next_batch(5)    
        X_mb = Variable(torch.from_numpy(X_mb))
        Z_mb = sample_Z(5, Dim) 
        Z_mb = Variable(torch.from_numpy(Z_mb.astype('float32')))
        M_mb = sample_M(5, Dim, p_miss)
        M_mb = Variable(torch.from_numpy(M_mb.astype('float32')))
        New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb
        
        samples1 = X_mb                
        samples5 = M_mb * X_mb + (1-M_mb) * Z_mb

        
        samples2 = generator(X_mb,Z_mb,M_mb)
        samples2 = M_mb * X_mb + (1-M_mb) * samples2        

        
        Z_mb = sample_Z(5, Dim) 
        Z_mb = Variable(torch.from_numpy(Z_mb.astype('float32')))
        New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb       
        samples3 = generator(X_mb,Z_mb,M_mb)
        samples3 = M_mb * X_mb + (1-M_mb) * samples3     

        
        Z_mb = sample_Z(5, Dim) 
        Z_mb = Variable(torch.from_numpy(Z_mb.astype('float32')))
        New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb       
        samples4 = generator(X_mb,Z_mb,M_mb)
        samples4 = M_mb * X_mb + (1-M_mb) * samples4 

        
        
        
        samples = torch.cat([samples5,samples2,samples3,samples4,samples1],0)
        samples = samples.data.numpy()
        fig = plot(samples)
        plt.savefig('MNIST_Impuation_output/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)



  0%|          | 0/10000 [00:00<?, ?it/s]

Iter: 0
Train_loss: 0.265
Test_loss: 0.2641



  1%|          | 99/10000 [00:06<11:10, 14.76it/s] 

Iter: 100
Train_loss: 0.06519
Test_loss: 0.06615



  2%|▏         | 199/10000 [00:13<10:44, 15.20it/s]

...

 98%|█████████▊| 9800/10000 [18:39<00:22,  8.75it/s]

Iter: 9800
Train_loss: 0.01612
Test_loss: 0.02744



 99%|█████████▉| 9900/10000 [18:51<00:11,  8.75it/s]

Iter: 9900
Train_loss: 0.01659
Test_loss: 0.02847



100%|██████████| 10000/10000 [19:03<00:00,  8.75it/s]
